## Import

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
assert gensim.models.word2vec.FAST_VERSION > -1
from gensim.models import Word2Vec
import datetime
import requests
import re
from bs4 import BeautifulSoup

## Data 준비하기

In [2]:
## < Data 준비하기 >
# 데이터 읽기
df_scholarship = pd.read_csv('dataset/scholarship_process.csv') # 장학금 데이터
# df_contest = pd.read_csv('contest.csv') # 공모전 데이터

# 인코딩
id_to_name = pd.Series(df_scholarship.scholarship_name.values, index = df_scholarship.idx.values)
name_to_id = pd.Series(df_scholarship.idx.values, index = df_scholarship.scholarship_name).to_dict()

df_scholarship.head(5)

,Unnamed: 0,idx,scholarship_name,foundation,scholarship_type,근로장학,기숙사/주거지원,기타장학,대출지원,연수/연구장학,...,대학원생,전문대생,중학생,청년,초등학생,선발 인원(명),선발인원규모,장학혜택(만원),장학혜택규모,memo
0,0,2000,2020년 포스코비전장학,포스코청암재단,장학금,0,0,0,0,0,...,0,0,0,0,0,99,대,1440,대,어려운 환경에서 올바른 품성을 가지고 학업에 정진하는 청소년들을 지원하고자 제철소 ...
1,2,2002,서원대 총동문회 장학 (2019년),서원대학교 총동문회,장학금,0,0,0,0,0,...,0,0,0,0,0,9,소,100,소,서원대학교 총동문회에서 총동문회비를 납부한 서원대 재학생을 지원하는 장학입니다.
2,3,2003,예술인 전세자금대출 (2019년),한국예술인복지재단,대출지원,0,0,0,1,0,...,0,0,0,0,0,0,인원미정,0,금액미정,한국예술인복지재단이 진행하는 예술인생활안정자금(융자) 전세자금대출 사업입니다.
3,4,2004,KRX DREAM 대학생 장학 (2020년),KRX 국민행복재단,장학금,0,0,0,0,0,...,0,1,0,0,0,40,중,600,대,KRX 국민행복재단에서 서울 및 부산 소재 대학에 재학 중인 학생을 지원하는 장학입니다.
4,10,2010,국가근로장학 (2020년 1학기),한국장학재단,근로장학,1,0,0,0,0,...,0,1,0,0,0,0,인원미정,0,금액미정,한국장학재단에서 시행하는 안정적인 학업여건 조성과 취업역량 제고를 위한 장학금입니다.


## 필요 없는 column 제거

In [3]:
df_scholarship = df_scholarship.drop(['Unnamed: 0', 'scholarship_type', 'target', '선발 인원(명)', '장학혜택(만원)', 'memo'], axis=1)
df_scholarship.head(5)

,idx,scholarship_name,foundation,근로장학,기숙사/주거지원,기타장학,대출지원,연수/연구장학,장학금,취업연계/산학장학,고등학생,기타/일반,대학생,대학원생,전문대생,중학생,청년,초등학생,선발인원규모,장학혜택규모
0,2000,2020년 포스코비전장학,포스코청암재단,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,대,대
1,2002,서원대 총동문회 장학 (2019년),서원대학교 총동문회,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,소,소
2,2003,예술인 전세자금대출 (2019년),한국예술인복지재단,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,인원미정,금액미정
3,2004,KRX DREAM 대학생 장학 (2020년),KRX 국민행복재단,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,중,대
4,2010,국가근로장학 (2020년 1학기),한국장학재단,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,인원미정,금액미정


## train, test set 만들기

In [4]:
# < train set, test set 만들기 >
df_scholarship_train, df_scholarship_test = train_test_split(df_scholarship, random_state = 15688, test_size=0.30)

## 장학금 타입, 지원 대상, 선발인원규모, 장학혜택규모가 같은 장학금끼리 묶기

In [5]:
# 비교할 때 제외할 열들의 이름 리스트
def data_splitter(df):
    exclude_cols = ['idx', 'scholarship_name', 'foundation']

    grouped_cols = [col for col in df_scholarship.columns if col not in exclude_cols]  # 제외할 열을 제외한 열들의 리스트

    df_grouped = df_scholarship.groupby(grouped_cols).apply(lambda x: x.idx.tolist())
    
    # 결과값을 str 타입으로 저장한 이중 배열
    array_of_groups = df_grouped.apply(lambda x: [str(val) for val in x]).tolist()
    return array_of_groups

In [6]:
pd.options.mode.chained_assignment = None
scholarship_ls = data_splitter(df_scholarship_train)

## 데이터 랜덤으로 섞고, 유효한 데이터 골라내기

In [7]:
for scholarships in scholarship_ls:
    random.shuffle(scholarships)

In [8]:
filtered_array = [group for group in scholarship_ls if len(group) > 1]

In [9]:
filtered_array

[['3193', '4407'],
 ['2190', '3401'],
 ['2135', '3192'],
 ['3621', '2465'],
 ['4370', '3143'],
 ['2237', '2181'],
 ['5531', '5533', '5532'],
 ['2529', '5092'],
 ['5709', '3827'],
 ['5521', '5585'],
 ['5582', '6113', '5937', '6163'],
 ['2624', '4540'],
 ['2112', '3157'],
 ['4886', '2461'],
 ['3611', '4660'],
 ['2487', '4889', '3653'],
 ['2464', '4486'],
 ['5049',
  '3572',
  '3573',
  '2613',
  '5851',
  '3838',
  '2515',
  '3370',
  '6194',
  '2422',
  '4903'],
 ['5633',
  '5086',
  '6197',
  '3209',
  '3739',
  '6257',
  '2082',
  '6012',
  '2640',
  '6006',
  '5064',
  '5325',
  '6007',
  '3294',
  '2097',
  '4582',
  '3077'],
 ['5502', '2021'],
 ['5692', '4448'],
 ['6207',
  '2140',
  '4148',
  '4651',
  '2521',
  '6043',
  '5254',
  '3089',
  '4361',
  '5091',
  '2026',
  '4193',
  '4829',
  '6208',
  '4879',
  '2318',
  '4994'],
 ['6183',
  '3712',
  '4548',
  '2024',
  '3748',
  '5500',
  '2366',
  '6071',
  '4837',
  '4197',
  '2290'],
 ['4306', '4932', '3860', '3658'],
 ['2363'

## Model 생성 및 학습

In [15]:
# 모델 생성
model = Word2Vec(
    sentences=filtered_array,  # 전처리된 리스트를 파라미터로 사용
    vector_size=100,  # hidden layer 사이즈
    window=5,  # 주변 단어 창 크기
    min_count=1,  # 최소 등장 횟수
    workers=4,  # 사용할 CPU 스레드 수
    sg=1,  # Skip-gram 모델 사용
    hs=0,  # Negative sampling 사용
    negative=5,  # Negative sampling에서 사용할 negative 단어의 개수
    epochs=5  # 훈련 에포크 수
)

# 모델 저장
# model.save('item2vec_20230516')


# < 모델 불러오기 >
# warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

# model = Word2Vec.load('item2vec_20230407')

In [16]:
print(list(model.wv.index_to_key))

['3183', '4150', '2176', '2748', '3626', '3964', '3258', '3958', '4401', '2949', '3802', '4472', '5444', '3500', '2031', '4744', '6034', '5753', '2141', '3518', '3120', '3737', '5379', '3757', '4323', '4693', '5881', '4864', '2499', '2209', '3218', '4422', '3270', '2277', '5981', '2829', '5949', '4165', '2404', '4924', '5788', '3103', '2245', '2174', '2293', '4635', '5418', '3772', '3150', '5498', '5275', '4574', '2978', '5569', '5011', '5940', '3719', '4545', '2833', '4371', '3136', '2989', '2758', '5297', '4139', '5276', '2169', '4535', '2154', '5959', '2764', '3174', '3623', '2403', '3439', '3508', '5206', '4905', '4244', '2055', '3196', '4449', '5874', '2646', '3571', '5442', '2391', '2767', '3232', '4600', '5090', '5457', '3346', '2325', '5550', '4398', '5958', '4499', '2689', '4745', '2150', '4468', '2235', '5551', '4351', '5992', '4204', '5025', '6003', '3292', '4760', '3855', '5288', '3980', '2720', '3160', '3822', '4456', '2275', '2850', '5689', '2719', '6199', '6158', '3406',

## 추천

In [17]:
def recommender(positive_list=None, negative_list=None, topn=20):
    recommend_scholarship_ls = []
    
    for scholarshipId, prob in model.wv.most_similar_cosmul(positive=positive_list, negative=negative_list, topn=topn):
        recommend_scholarship_ls.append(scholarshipId)
    return recommend_scholarship_ls

In [18]:
# recommender
reco = [2016] # 검색하고 싶은 idx
reco_str = [str(num) for num in reco]
recommandations = recommender(positive_list=reco_str, topn=5)
reco_int = [int(value) for value in recommandations] # str -> int
print(reco_int)

[4723, 2685, 2421, 4440, 2336]


In [19]:
# 입력 데이터
filtered_scholarship_names = df_scholarship.loc[df_scholarship['idx'].isin(reco), 'scholarship_name'].tolist()
print(filtered_scholarship_names)

['경희대 이수병 장학 (2020년)']


In [20]:
# 출력 데이터
filtered_scholarship_names = df_scholarship.loc[df_scholarship['idx'].isin(reco_int), 'scholarship_name'].tolist()
print(filtered_scholarship_names)

['서강대 서강예수회 장학 (2020년 1학기)', '보호종료청년 자립정착 지원', '종로 문화기획자 양성 프로그램', '부산 사상구 장학 (2022년)', '파주시 청년 취업성공 참여자 모집']
